# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
!pip install pymongo

In [16]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [17]:
# ENTRAMOS DENTRO DE NUESTRA DATABASE IRONHACK DE MONGO (para luego especificarle que collecion coger)
db = client.get_database("ironhack")

In [18]:
# Una vez descargado el zip, guardado en mi ordenador descomprimido: lo subimos a mongo dentro de ironhack y lo llamos en python
# de la siguiente manera
collection = db.get_collection("companies")

In [24]:
#para que no imprima todos los datos lo meto en una variable. pero funciona jeje sacao todas las compañias que tengan ese nombre
companies_Babelgum = collection.find_one({"name":'Babelgum'})

In [26]:
collection.find_one({})

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [43]:
'''
1. Llamamos a la key que queremos con su condicion (compañias con MAS de 5000 empleados)
2. en el segundo corchete ponemos como queremos que nos lo devuelva:
    - que solo nos imprima el nº empleados y el nombre, tenemos que poner que no nos imprima el id xq siempre se imprime x defecto)
    - RECORDATORIO!!: 1--> que imprima  // 0--> que NO imprima
3. utilizamos .sort para ordenarlo: 
    - ponemos la variable por la que queremos que nos lo ordene (en este caso n1 de empleados)
    - (1) --> de menor a mayor   //   -1--> de mayor a menor
4. ponemos el limite de cuatas empresas quiere que nos imprima
''' 

employees = collection.find({'number_of_employees':{"$gt": 5000}}, {"number_of_employees": 1, "name": 1, "_id" : 0}).sort("number_of_employees", 1).limit(20)

In [44]:
list(employees)

[{'name': 'Nintendo', 'number_of_employees': 5080},
 {'name': 'Hexaware Technologies', 'number_of_employees': 5200},
 {'name': 'Facebook', 'number_of_employees': 5299},
 {'name': 'OpenText', 'number_of_employees': 5300},
 {'name': 'CPM Braxis', 'number_of_employees': 5400},
 {'name': 'LSI', 'number_of_employees': 5400},
 {'name': 'Microchip Technologies', 'number_of_employees': 5500},
 {'name': 'Mediaset', 'number_of_employees': 5729},
 {'name': 'Mindray Medical International', 'number_of_employees': 5763},
 {'name': 'Dentsu', 'number_of_employees': 6000},
 {'name': 'Atmel', 'number_of_employees': 6000},
 {'name': 'Tata Communications', 'number_of_employees': 6000},
 {'name': 'Baidu', 'number_of_employees': 6000},
 {'name': 'Acxiom', 'number_of_employees': 6200},
 {'name': 'Acxiom', 'number_of_employees': 6200},
 {'name': 'SRA International', 'number_of_employees': 6400},
 {'name': "Moody's", 'number_of_employees': 6800},
 {'name': 'ManTech', 'number_of_employees': 7000},
 {'name': 'Ad

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [56]:
# ponmgo orden de mayor a menos para ver que se imprimen las de 2005 tanto como las del 2000
# y pongo un limite para que no se impriman todas que son bastantes 

founded_year = collection.find({'founded_year':{"$gt": 1999} and {"$lt": 2006}}, {"founded_year": 1, "name": 1, "_id" : 0}).sort("founded_year", -1).limit(25)

In [57]:
list(founded_year)

[{'name': 'Revision3', 'founded_year': 2005},
 {'name': 'LifeLock', 'founded_year': 2005},
 {'name': 'iSkoot', 'founded_year': 2005},
 {'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'Slide', 'founded_year': 2005},
 {'name': 'Jingle Networks', 'founded_year': 2005},
 {'name': 'Swivel', 'founded_year': 2005},
 {'name': 'ClipBlast!', 'founded_year': 2005},
 {'name': 'Instructables', 'founded_year': 2005},
 {'name': 'Jangl SMS', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Bebo', 'founded_year': 2005},
 {'name': 'Jajah', 'founded_year': 2005},
 {'name': 'TripHub', 'founded_year': 2005},
 {'name': 'YouTube', 'founded_year': 2005},
 {'name': 'Wesabe', 'founded_year': 2005},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'PBworks', 'founded_year': 2005},
 {'name': 'RockYou', 'founded_year': 2005},
 {'name': 'Newsvine', 'founded_year': 2005},
 {'name': 'Meebo', 'founded_year': 2005},
 {'name': 'Netvibes', '

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [85]:
collection.find({'ipo.valuation_amount': {"$gt": 100000000}, 'founded_year':{"$lt": 2010}}, {"ipo.valuation_amount": 1, "name": 1, "_id" : 0})

In [86]:
list(_)

[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Brightcove', 'ipo': {'valuation_amount': 290000000}},
 {'name': 'KIT digital', 'ipo': {'valuation_amount': 235000000}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'OpenTable', 'ipo': {'valuation_amount': 1050000000}},
 {'name': 'ChannelAdvisor', 'ipo': {'valuation_amount': 287000000}},
 {'name': 'Jive Software', 'ipo': {'valuation_amount': 1000000000}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000}},
 {'name': 'Wix', 'ipo': {'valuation_amount': 750000000}},
 {'name': 'Shutterfly', 'ipo': {'valuation_amount': 350000000}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000}},
 {'name': 'Salesforce', 'ipo': {'

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [71]:
# pongo solamente que se imprima ell nombre, el año y el nº de employees porque sino hay mchos datos

collection.find({'number_of_employees': {"$lt": 1000}, 'founded_year':{"$lt": 2005}}, {'number_of_employees':1, "founded_year": 1, "name": 1, "_id" : 0}).sort("number_of_employees", -1).limit(10)

In [72]:
list(_)

[{'name': 'Infinera Corporation',
  'number_of_employees': 974,
  'founded_year': 2000},
 {'name': 'NorthPoint Communications Group',
  'number_of_employees': 948,
  'founded_year': 1997},
 {'name': '888 Holdings', 'number_of_employees': 931, 'founded_year': 1997},
 {'name': 'Forrester Research',
  'number_of_employees': 903,
  'founded_year': 1983},
 {'name': 'SonicWALL', 'number_of_employees': 900, 'founded_year': 1991},
 {'name': 'Webmetrics', 'number_of_employees': 900, 'founded_year': 1999},
 {'name': 'Cornerstone OnDemand',
  'number_of_employees': 881,
  'founded_year': 1999},
 {'name': 'Mozilla', 'number_of_employees': 800, 'founded_year': 1998},
 {'name': 'Buongiorno', 'number_of_employees': 800, 'founded_year': 1999},
 {'name': 'Yelp', 'number_of_employees': 800, 'founded_year': 2004}]

### 6. All the companies that don't include the `partners` field.

In [77]:
# no debe de haber ninguna compañia sin partners fields, porque hemos puesto aquellas que NO tengan partners y no sale ninguna...

collection.find_one({"partners": {"$exists": False}})

### 7. All the companies that have a null type of value on the `category_code` field.

In [80]:
# si que hay valores nulos pero no entiendo por que no me sale 

list(collection.find({"category_code": {"$regex": ".*null.*"}}, {"name": 1, "_id": 0}))

[]

In [83]:
# All the companies whose name contains the substring "hola" (el del readme)
hola = collection.find({"category_code": {"$regex": ".*Hola.*"}}, {"name": 1, "category_code":1, "_id": 0})

In [84]:
list(hola)

[]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [87]:
collection.find({'number_of_employees':{"$gt": 99} and {"$lt": 1000}}, {"number_of_employees": 1, "name": 1, "_id" : 0})

### 9. Order all the companies by their IPO price in a descending order.

In [102]:
collection.find_one({}).keys()


dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [139]:
#ESTE INCAPAZ DE SACARLO!!!! :(
collection.find_one('ipo').sort('ipo.price', -1)

AttributeError: 'NoneType' object has no attribute 'sort'

In [124]:
collection.find_one({'ipo.valuation_amount': {"$exists": True}}, {'ipo.valuation_amount':1, 'name':1, '_id':0}).sort('ipo.valuation_amount', -1)

AttributeError: 'dict' object has no attribute 'sort'

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [ ]:
collection.find('number_employees').sort('number_employees', -1).limit(10)

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [137]:
collection.find({'founded_month': {"$gt": 5} and {"$lt": 13}, 'founded_month':1, "name": 1, "_id" : 0}).sort("founded_month", -1).limit(10)

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [130]:
collection.find({'acquisition_amount': {"$gt": 1000000}, 'founded_year':{"$lt": 2000}})

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [141]:
collection.find({'acquisition.acquired_year': {"$gt": 2010}}, {'name':1, 'aquisition':1, '_id' : 0}).sort('acquisition.price_amount', -1)

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [148]:
collection.find({'founded_year':1, 'name':1, '_id': 0}).sort('founded_year', 1)

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [158]:
# ver otra vez!!!
collection.find_one({'acquisition.acquired_day': {"$lt": 8}}, {'name':1, 'acquisition.acquired_day':1, 'acquisition.price_amount':1}).sort('acquisition.price_amount', -1).limit(10)


AttributeError: 'dict' object has no attribute 'sort'

In [ ]:
collection.find_one({"address line 2": {"$exists": True}})

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [167]:

category = collection.find({'category_code': {"$regex": ".*web.*"}, 'number_of_employees':{"$gt": 400}}, {'number_of_employees':1, "category_code": 1, "name": 1, "_id" : 0}).sort("number_of_employees", 1)

In [168]:
list(category)

[{'name': 'ITP Publishing Group',
  'category_code': 'web',
  'number_of_employees': 437},
 {'name': 'GARA Web Hosting',
  'category_code': 'web',
  'number_of_employees': 480},
 {'name': 'Central Reservations',
  'category_code': 'web',
  'number_of_employees': 510},
 {'name': 'Leapfrog Enterprises',
  'category_code': 'web',
  'number_of_employees': 541},
 {'name': 'OpenTable', 'category_code': 'web', 'number_of_employees': 550},
 {'name': 'zanox', 'category_code': 'web', 'number_of_employees': 600},
 {'name': 'MLB Advanced Media',
  'category_code': 'web',
  'number_of_employees': 600},
 {'name': 'Consumers Union',
  'category_code': 'web',
  'number_of_employees': 600},
 {'name': 'Citrix Online', 'category_code': 'web', 'number_of_employees': 700},
 {'name': 'Toluna', 'category_code': 'web', 'number_of_employees': 700},
 {'name': 'Mozilla', 'category_code': 'web', 'number_of_employees': 800},
 {'name': 'Gumtree', 'category_code': 'web', 'number_of_employees': 1000},
 {'name': 'Home

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [173]:
collection.find({'acquisition_amount': {"$gt": 10000000}, 'acquisition.price_currency_code':{"$regex": ".*EUR.*"}})


In [174]:
# al parecer no hay ninguna empresa que cumpla ambas condiciones
list(_)

[]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [185]:
collection.find({'acquisition.acquired_month':{"$lt": 5}}, {"acquisition": 1, "name": 1, "_id" : 0}).limit(10)

In [178]:
list(_)

[{'name': 'StumbleUpon',
  'acquisition': {'price_amount': 29000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2009/04/13/ebay-unacquires-stumbleupon/',
   'source_description': "StumbleUpon Beats Skype In Escaping EBay's Clutches",
   'acquired_year': 2009,
   'acquired_month': 4,
   'acquired_day': None,
   'acquiring_company': {'name': 'StumbleUpon', 'permalink': 'stumbleupon'}}},
 {'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'Veoh',
  'acquisition': {'price_amount': None,
   'price_currenc

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [207]:
# Pregubntaarrr!! 

collection.find_one({'founded_year':{"$gt": 2000} and {"$lt": 2010},  'investments.acquisition': {"$regex": ".*null.*"}}, {'investments.acquisition': 1, "name": 1, "_id" : 0})

### 20. All the companies that have been 'deadpooled' after the third year.

In [197]:
collection.find_one({"deadpooled_year": {"$gt": 3}}, {'name':1, "deadpooled_year":1, '_id': 0})


{'name': 'Omnidrive', 'deadpooled_year': 2008}